![](imgs/ensemble.png)

In [1]:
from elastic_db.elasticsearch import elastic_conection, NLPmodelIndex

es = elastic_conection()
index = NLPmodelIndex(es=es, workspace_id="dc1e7b3d-9137-4a20-a99c-d0d2029ef170")
index.index_alias

ModuleNotFoundError: No module named 'elastic_db'

In [3]:
es.get(index=index.index_name, id=index.workspace_id)

{'_index': 'nlp_model-dc1e7b3d-9137-4a20-a99c-d0d2029ef170-all-10022021085104',
 '_type': '_doc',
 '_id': 'dc1e7b3d-9137-4a20-a99c-d0d2029ef170-all',
 '_version': 3,
 '_seq_no': 425,
 '_primary_term': 1,
 'found': True,
 '_source': {'workspace_id': 'dc1e7b3d-9137-4a20-a99c-d0d2029ef170-all',
  'customer_id': 'yara',
  'recipe': {'model_kind': 'BM25', 'BM25': {'b': 0.75, 'k1': 1.2}},
  'cnn_accuracy': 0.8552188277244568,
  'xgb_accuracy': 0.3939393939393939,
  'svm_accuracy': 0.7845117845117845,
  'logit_accuracy': 0.6531986531986532,
  'accuracies': {'cnn_accuracy': 0.8383838534355164,
   'xgb_accuracy': 0.3939393939393939,
   'svm_accuracy': 0.7845117845117845,
   'logit_accuracy': 0.6531986531986532}}}

# Ler Modelos

In [29]:
from joblib import load
from tensorflow.keras.models import load_model

path = "/home/cloves/Desktop/omni_nlp_api"

def load_models(workspace_id):
    svm = load(path+'/models/polySVM/trained_models/svm-{workspace_id}.joblib'.format(workspace_id=workspace_id))
    logit = load(path+'/models/multinomialLogit/trained_models/logit-{workspace_id}.joblib'.format(workspace_id=workspace_id))
    cnn = load_model(path+'/models/cnn/trained_models/cnn-{workspace_id}.h5'.format(workspace_id=workspace_id))
    return svm, logit, cnn

svm, logit, cnn = load_models(index.workspace_id)

# Ler Dicionário de Intenções

In [34]:
import json 

file = "/home/cloves/Desktop/omni_nlp_api/feature_encoding/tfidf/resources/intents_dictionary-{workspace_id}.json".format(workspace_id=index.workspace_id)
with open(file, 'r') as f:
      intents_dictionary = json.load(f)

# Texto para tfidf

In [19]:
import numpy as np
import pickle

query = {
  "tokenizer" : "classic",
  "filter" : [
              "lowercase",
              "asciifolding",
              {"type": "stop", "stopwords": "_portuguese_"},
              {"type": "stemmer", "language": "brazilian"}],
  "text" : ""
}

file = "/home/cloves/Desktop/omni_nlp_api/feature_encoding/tfidf/resources/tf_idf_corpus_{workspace_id}.pickle".format(workspace_id=index.workspace_id)
tf_idf_corpus = pickle.load(open(file, "rb"))

def get_sentece_features(sentence, index, es):
    query["text"] = sentence
    result = es.indices.analyze(index=index.index_name, body=query)
    new_sentence = " ".join([token["token"] for token in result["tokens"]])
    sentence_features = tf_idf_corpus.transform([new_sentence]).toarray()
    return sentence_features 

# Ensemble

In [ ]:
# def ensemble_prob_vector(result_vectors):
#     for prob_vector in result_vectors:
#         acc
    

In [25]:
nlp_model_doc = es.get(index=index.index_alias, id=index.workspace_id)
accuracies = nlp_model_doc["_source"]["accuracies"]
accuracies

{'cnn_accuracy': 0.8518518805503845,
 'xgb_accuracy': 0.3939393939393939,
 'svm_accuracy': 0.7845117845117845,
 'logit_accuracy': 0.6531986531986532,
 'bm25_accuracy': 0.7272727272727273}

In [45]:
# Duvida_Formularo_Adesao_Previdencia
sentence = "dúvidas sobre o preenchimento do formulário"
sentence_features = get_sentece_features(sentence, index, es)
result_vectors = []
logit_result = logit.predict_proba(sentence_features) * accuracies["logit_accuracy"]
cnn_result = cnn.predict(sentence_features) * accuracies["cnn_accuracy"]
svm_result = svm.predict_proba(sentence_features) * accuracies["svm_accuracy"]
ensemble_result = logit_result + cnn_result + svm_result

In [46]:
intents_dictionary[str(np.argmax(ensemble_result))]

'Duvida_Formularo_Adesao_Previdencia'

In [49]:
np.argmax(ensemble_result)

36

In [54]:
ensemble_result[0][36]

0.37667835606515465

In [43]:
from models.BM25.classifier import find_intents_bm25

result = find_intents_bm25(index, sentence, es)
result

{'intents': [{'intent': 'Duvida_Formularo_Adesao_Previdencia',
   'confidence': 0.7550395709077156},
  {'intent': 'Bot_Capabilities', 'confidence': 0.7052722839057859},
  {'intent': 'Visao_Geral_Previdencia_Privada',
   'confidence': 0.6914305289932955},
  {'intent': 'Receber_Cartao_Convenio', 'confidence': 0.6419680728609294},
  {'intent': 'Direcao_Defensiva', 'confidence': 0.6315279295012705},
  {'intent': 'Veiculo_Problema', 'confidence': 0.6302607881582883},
  {'intent': 'Ex_Colaborador_Resgate_Previdencia',
   'confidence': 0.6246843877187425},
  {'intent': 'Detalhe_Sobre_Desconto', 'confidence': 0.5956599432193345},
  {'intent': 'Cartao_Bloqueado_SenhaInvalida',
   'confidence': 0.5478608096736808},
  {'intent': 'Licenca_Mais_Ferias', 'confidence': 0.5347608978072536}]}